In [26]:
from sentence_transformers import SentenceTransformer
from transformers import ElectraModel
import utils as keu # key_electra_utils.py 참고
from konlpy.tag import Hannanum
import pandas as pd
import ast
import re

model_new = SentenceTransformer(model_name_or_path='model/disc_book',tokenizer_path='model/tokenizer/')
model_old = SentenceTransformer(model_name_or_path='monologg/koelectra-base-v3-discriminator')

No sentence-transformers model found with name model/disc_book. Creating a new one with MEAN pooling.
No sentence-transformers model found with name /Users/yangwoolee/.cache/torch/sentence_transformers/monologg_koelectra-base-v3-discriminator. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /Users/yangwoolee/.cache/torch/sentence_transformers/monologg_koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly

In [2]:
raw_data = pd.read_csv('data/raw/raw_book_info_list.csv',index_col=0)

raw_data.head(2)

,book_title,book_toc,book_intro,publisher
0,한 권으로 끝내는 메타버스 크리에이터,"['메타버스란', '왜 메타버스인가', '메타버스의 유형을 알아보자', '메타버스 ...",[],[]
1,Do it! 점프 투 파이썬: 라이브러리 예제 편,"['', '텍스트 다루기', '문자열을 줄여 표시하려면 textwrap shorte...",['이 책은 Do it 점프 투 파이썬 의 박응용 저자가 그동안 수많은 독자에게 받...,['실무에서 자주 쓰는 파이썬 라이브러리는 다 있다 필수 파이썬 라이브러리 개 엄선...


## [데이터 전처리] 단어 키워드 추출에 맞는 Input 데이터 생산

### 1. 도서 정보를 하나의 string으로 넣기

In [75]:
book_info :str =keu.merge_series_to_str(raw_data.iloc[2323])

변환한 도서정보 :  한 권으로 끝내는 딥러닝 텐서플로


### 2. 일부 영단어를 한글로 변환하기

In [76]:
englist = pd.read_csv('preprocess/englist.csv')
book_info_trans = keu.trans_eng_to_han(book_info,englist=englist)

### 3. 문장 내 영단어 제거 및 영단어 모아두기

In [77]:
book_info_han = keu.find_han(' '.join(book_info_trans))
book_info_eng = keu.find_eng(' '.join(book_info_trans))

### Konlpy로 명사만 추출하기

In [78]:
from konlpy.tag import Hannanum

han = Hannanum()

# str or list에 따른 tokenizing 방법
if type(book_info_han) == list :
    han_nouns = han.nouns(' '.join(book_info_trans))
    
else :
    print('book_info type must be list.')

### Sentence Transformer로 유사도 검색

In [79]:
keu.key_extraction(han_nouns,model_old)

,0
딥,0.894649
하이퍼파라미터,0.785934
완전연결,0.777877
신경망,0.771819
딥러닝,0.765136
사용,0.761644
텐서플,0.758479
강화학습,0.752364
텐서플로,0.748740
아키텍처,0.727021


In [80]:
keu.key_extraction(han_nouns,model_new,min_rank=20)

,0
딥,0.887673
딥러닝,0.874081
사용,0.867020
텐서플,0.848507
하이퍼파라미터,0.839869
완전연결,0.814461
아키텍처,0.811337
텐서플로,0.787370
신경망,0.779551
강화학습,0.768536
